In [1]:
import cv2
import numpy as np
import pandas as pd
import os
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import tensorflow.keras as K
import tensorflow.keras.backend as Kback
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
import tensorflow as tf

# from google.colab import drive
# drive.mount('/content/drive')

# Model Training

In [2]:
def f1_score(y_true, y_pred): #taken from old keras source code
    true_positives = Kback.sum(Kback.round(Kback.clip(y_true * y_pred, 0, 1)))
    possible_positives = Kback.sum(Kback.round(Kback.clip(y_true, 0, 1)))
    predicted_positives = Kback.sum(Kback.round(Kback.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + Kback.epsilon())
    recall = true_positives / (possible_positives + Kback.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+Kback.epsilon())
    return f1_val

METRICS = [
      "accuracy",
      K.metrics.Precision(name='precision'),
      K.metrics.Recall(name='recall'),
      K.metrics.AUC(name='auc'),
      f1_score
]

In [3]:
input_layer = K.Input(shape=(160,160,3))
resnet_model = K.applications.MobileNetV2(include_top = False, weights = "imagenet", input_tensor = input_layer)
for layer in resnet_model.layers:
    layer.trainable = False
for i, layer in enumerate(resnet_model.layers):
    print(i, layer.name, "-", layer.trainable)

0 input_1 - False
1 Conv1 - False
2 bn_Conv1 - False
3 Conv1_relu - False
4 expanded_conv_depthwise - False
5 expanded_conv_depthwise_BN - False
6 expanded_conv_depthwise_relu - False
7 expanded_conv_project - False
8 expanded_conv_project_BN - False
9 block_1_expand - False
10 block_1_expand_BN - False
11 block_1_expand_relu - False
12 block_1_pad - False
13 block_1_depthwise - False
14 block_1_depthwise_BN - False
15 block_1_depthwise_relu - False
16 block_1_project - False
17 block_1_project_BN - False
18 block_2_expand - False
19 block_2_expand_BN - False
20 block_2_expand_relu - False
21 block_2_depthwise - False
22 block_2_depthwise_BN - False
23 block_2_depthwise_relu - False
24 block_2_project - False
25 block_2_project_BN - False
26 block_2_add - False
27 block_3_expand - False
28 block_3_expand_BN - False
29 block_3_expand_relu - False
30 block_3_pad - False
31 block_3_depthwise - False
32 block_3_depthwise_BN - False
33 block_3_depthwise_relu - False
34 block_3_project - Fal

In [4]:
def fft_2d(feature_map):
    feature_map = tf.cast(feature_map, tf.complex64)
    X1 = tf.signal.fft2d(feature_map)
    X1 = tf.abs(X1)
    return X1   

inputs = K.layers.Input(shape=(160,160,3), name="image") 
resnet50 = resnet_model(inputs)
fft_layer = K.layers.Lambda(lambda x: fft_2d(x))(resnet50)
fft_flat = K.layers.GlobalAveragePooling2D()(fft_layer)
output = K.layers.Dense(20, activation='softmax')(fft_flat)

model = K.Model(inputs=inputs, outputs=fft_flat)
optimizer = K.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, clipnorm=1.0)
model.compile(loss=["categorical_crossentropy"], metrics=METRICS, optimizer = optimizer)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 image (InputLayer)          [(None, 160, 160, 3)]     0         
                                                                 
 mobilenetv2_1.00_224 (Funct  (None, 5, 5, 1280)       2257984   
 ional)                                                          
                                                                 
 lambda (Lambda)             (None, 5, 5, 1280)        0         
                                                                 
 global_average_pooling2d (G  (None, 1280)             0         
 lobalAveragePooling2D)                                          
                                                                 
Total params: 2,257,984
Trainable params: 0
Non-trainable params: 2,257,984
_________________________________________________________________


C:\Users\SAPTARSHI PANI\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [5]:
import os
import random
import numpy as np

Y_train = []
X_train = []
for i in range(1,21):
  folder = "E:/data/train/"+str(i)+"/"
  for image in os.listdir(os.path.join(folder)):
    I = []
    img = cv2.imread(os.path.join(folder,image))
    img = cv2.resize(img, (160, 160))
    img = img/255
    I.append(img)
    I = np.asarray(I)
    X_train.append(np.asarray(model.predict(I)[0]))
    Y_train.append(i)

Y_train = np.asarray(Y_train)
X_train = np.asarray(X_train)

Y_test = []
X_test = []
for i in range(1,21):
  folder = "E:/data/test/"+str(i)+"/"
  for image in os.listdir(os.path.join(folder)):
    I = []
    img = cv2.imread(os.path.join(folder,image))
    img = cv2.resize(img, (160, 160))
    img = img/255
    I.append(img)
    I = np.asarray(I)
    X_test.append(np.asarray(model.predict(I)[0]))
    Y_test.append(i)

Y_test = np.asarray(Y_test)
X_test = np.asarray(X_test)

shuffler = np.random.permutation(len(X_train))
X_train = X_train[shuffler]
Y_train = Y_train[shuffler]




 

 





#Load model
# model_xgb_2 = xgb.Booster()
# model_xgb_2.load_model("model.json")

1/1 [==============================] - 0s 33ms/step


1/1 [==============================] - 0s 34ms/step


1/1 [==============================] - 0s 36ms/step


1/1 [==============================] - 0s 38ms/step


1/1 [==============================] - 0s 38ms/step


...

1/1 [==============================] - 0s 32ms/step


1/1 [==============================] - 0s 35ms/step


1/1 [==============================] - 0s 33ms/step


1/1 [==============================] - 0s 34ms/step


1/1 [==============================] - 0s 36ms/step


In [6]:
shuffler = np.random.permutation(len(X_test))
X_test = X_test[shuffler]
Y_test = Y_test[shuffler]

In [7]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
Y_train = le.fit_transform(Y_train)

In [8]:
#XGBoost as classifier using the features of CNN+FFT
import xgboost as xgb
my_model = xgb.XGBClassifier(n_estimators = 400, learning_rate = 0.1, max_depth = 3)
my_model.fit(X_train, Y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=3, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=400, n_jobs=None, num_parallel_tree=None,
              objective='multi:softprob', predictor=None, ...)

In [9]:
# Predicting the Test set results
y_pred = my_model.predict(X_test)

In [10]:
# Making the Confusion Matrix
cm = confusion_matrix(Y_test, y_pred)
disp = ConfusionMatrixDisplay(cm,display_labels=['1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19','20'])

In [11]:
Y_test = le.fit_transform(Y_test)

In [12]:
#Accuracy
from sklearn.metrics import accuracy_score
print("Testing accuracy:")
print(accuracy_score(Y_test, y_pred))

Testing accuracy:
0.791


In [13]:
cm

array([[ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0],
       [62,  2,  0,  0,  0,  0,  0,  1,  0,  0,  2,  0,  0,  5,  0,  0,
         0,  0, 28,  0,  0],
       [ 8, 67,  0,  0,  0,  0,  0,  1,  0,  0,  2,  0,  0,  1,  0,  0,
         0,  0, 21,  0,  0],
       [ 1,  0, 73,  0,  0,  0,  0, 16,  1,  3,  0,  6,  0,  0,  0,  0,
         0,  0,  0,  0,  0],
       [ 0,  0,  0, 77,  5,  0,  0,  0,  0,  0,  0,  1, 16,  0,  0,  0,
         0,  0,  0,  1,  0],
       [ 0,  0,  0,  6, 89,  0,  0,  0,  0,  0,  0,  2,  2,  0,  0,  1,
         0,  0,  0,  0,  0],
       [ 0,  1,  0,  2,  2, 94,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  1,  0,  0,  0],
       [ 1,  0,  0,  0,  0,  0, 85,  0,  0,  0,  0,  0,  0,  0,  0, 11,
         1,  2,  0,  0,  0],
       [ 0,  0, 14,  0,  0,  0,  0, 64, 13,  5,  0,  4,  0,  0,  0,  0,
         0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  2, 93,  1,  0,  4,  0,  0,  0,  0,
         0,  0,  0,

In [14]:
#F1_score
from sklearn.metrics import f1_score
print("Testing F1-score")
print(f1_score(Y_test, y_pred, pos_label='positive', average='macro'))

Testing F1-score
0.7924264878506483


C:\Users\SAPTARSHI PANI\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1379: UserWarning: Note that pos_label (set to 'positive') is ignored when average != 'binary' (got 'macro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn(


In [15]:
#Precision
from sklearn.metrics import precision_score
print("Testing Precision:")
print(precision_score(Y_test, y_pred, pos_label='positive', average='macro'))
#Recall
from sklearn.metrics import recall_score
print("Testing Recall:")
print(recall_score(Y_test, y_pred, pos_label='positive', average='macro'))

Testing Precision:
0.8021855266215854
Testing Recall:
0.791


C:\Users\SAPTARSHI PANI\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1379: UserWarning: Note that pos_label (set to 'positive') is ignored when average != 'binary' (got 'macro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn(
C:\Users\SAPTARSHI PANI\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1379: UserWarning: Note that pos_label (set to 'positive') is ignored when average != 'binary' (got 'macro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn(


In [16]:
# save to JSON
my_model.save_model("model_1.json")
# save to text format
my_model.save_model("model_1.txt")

In [17]:
X_train.shape

(10000, 1280)